# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export 
import httpx
import zipfile
from fastcore.all import *
import pandas as pd

In [ ]:
#| hide 
df = pd.read_csv("company1.csv", index_col=0)
df.head()

,TATAMOTORS,ASHOKLEY,OLECTRA,FORCEMOT,SMLISUZU
Financial Year 2024,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....
Financial Year 2023,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....,https://www.bseindia.com/stockinfo/AnnPdfOpen....
Financial Year 2022,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...
Financial Year 2021,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...
Financial Year 2020,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...


In [ ]:
#| export
def make_dirs(base, df):
    base.mkdir(exist_ok=True)
    for c in df.columns:
        dir = base/c
        dir.mkdir(exist_ok=True)
        for i in df.index:
            if "Right " not in i:
                (dir/i).mkdir(exist_ok=True)

In [ ]:
make_dirs(Path("result"), df)

In [ ]:
df.iloc[0]["TATAMOTORS"]

'https://www.bseindia.com/stockinfo/AnnPdfOpen.aspx?Pname=0913b647-b205-4a3c-ad9b-8c493f97f972.pdf'

In [ ]:
col, row = df.columns, df.index
col, row, col[0], row[0]

(Index(['TATAMOTORS', 'ASHOKLEY', 'OLECTRA', 'FORCEMOT', 'SMLISUZU'], dtype='object'),
 Index(['Financial Year 2024', 'Financial Year 2023', 'Financial Year 2022',
        'Financial Year 2021', 'Financial Year 2020', 'Financial Year 2019',
        'Financial Year 2018', 'Financial Year 2017', 'Financial Year 2016',
        'Financial Year 2015', 'Financial Year 2014', 'Financial Year 2013',
        'Financial Year 2012', 'Financial Year 2011', 'Right Issue',
        'Financial Year 2010'],
       dtype='object'),
 'TATAMOTORS',
 'Financial Year 2024')

In [ ]:
#| export 
def download_url(url, fn):
    """
    download given url and write it a given fn
    """
    res = httpx.get(url)
    if res.status_code == 200:
        with open(fn, 'wb') as f:
            fn.write_bytes(res.content)
        return True
    else:
        print(f"Error downloading: Status code {res.status_code}")
        return False

In [ ]:
#| hide
!mkdir t

In [ ]:
url = "https://archives.nseindia.com/annual_reports/AR_ASHOKLEY_2009_2010_18082010121500.zip"
download_url(url, Path("t/temp.zip"))

True

In [ ]:
#| export
def rn_zip_extract(zipf, fn):
    """
    Donwload and extract the given files with filename fn 
    """
    home = Path(zipf).parent
    with zipfile.ZipFile(zipf, 'r') as zip_ref:
        f = zip_ref.namelist()[0]
        zip_ref.extractall(path=home)
        fi = home/f
        fi.rename(fn)


In [ ]:
rn_zip_extract(Path("t/temp.zip"), Path("t/i.pdf" ))

In [ ]:
#| hide
!ls t
!rm -rf t

i.pdf  temp.zip


In [ ]:
#| export 
def download_rename(url, fn, dir):
    """
    download given url and rename the file only handles zip and pdf
    """
    fn, dir = Path(fn), Path(dir)
    dir.mkdir(exist_ok=True)
    fn = dir/fn
    if ".zip" in url:
        zip = dir/"temp.zip"
        download_url(url, zip)
        rn_zip_extract(zip, fn)
        Path(dir/"temp.zip").unlink()
    elif ".pdf" in url:
        download_url(url, fn)

In [ ]:
download_rename(url, "i.pdf", "t")

In [ ]:
#| export 
from pdf2image import convert_from_path
from PIL.PpmImagePlugin import PpmImageFile
pdf2img = lambda x : convert_from_path(str(x), dpi=250)

In [ ]:
im = pdf2img(Path("t/i.pdf"))
assert type(im) == list
assert type(im[0]) == PpmImageFile

In [ ]:
#| export 
def save_md(fn: str, txt: str):
    """
    Writes the given text to a markdown file as binary (UTF-8 encoded).
    Parameters:
    fn (str): Filename or path to save the markdown file.
    txt (str): Content to write to the file.
    """
    try:
        # Open the file in binary write mode and encode the text to bytes
        with open(fn, 'wb') as f:
            f.write(txt.encode('utf-8'))
    except Exception as e:
        print(f"Error writing to file {fn}: {e}")
    

In [ ]:
save_md("temp.md", "yo yo")
p = Path("temp.md")
assert p.is_file()

In [ ]:
!rm -rf temp.md

In [ ]:
#| export
import math 
from PIL import Image
def qwen_resize(im:Image.Image, limit:int=1280*28*28, fn:str="temp.png" ):
    """
    check if the image of size `1280*28*28` which is the upper limit of image can be processed by 
    **QWEN VLM**.
    """
    w, h = im.size

    sf = math.sqrt( limit/(w*h))
    if sf < 1.:
        nw = int(w * sf)
        nh = int(h * sf)
        im = im.resize((nw,nh), Image.BICUBIC)

    im.save(fn)

In [ ]:
qwen_resize(im[0])
img = Image.open("temp.png")
assert img.size[0] < im[0].size[0] and img.size[1] < im[0].size[1]

In [ ]:
#| hide
!rm -rf t temp.png

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()